In [1]:
%matplotlib notebook

In [2]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.linear_model
from datetime import datetime
from sklearn_pandas import DataFrameMapper
import warnings
import timeit
from collections import defaultdict
import tabulate
import time
import GPy
from IPython.display import display

timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

matplotlib.style.use('ggplot')

RANDOM_SEED = 33

## Reading 2011-2014 as training, 2015 as validation, 2016 as test

In [3]:
NAMES = ('WBAN_number', 'UTC_date', 'UTC_time', 'LST_date', 'LST_time', 'CRX_version',
        'longitude', 'latitude', 'air_temperature', 'precepitation', 'solar_radiation',
        'SR_flag', 'surface_temperature', 'ST_type', 'ST_flag', 'relative_humidity', 'RH_flag',
        'soil_moisture', 'soil_temperature', 'wetness', 'wet_flag', 'wind', 'wind_flag')

TRAIN_YEARS = range(2011, 2015)
VALIDATION_YEAR = 2015
TEST_YEAR = 2016
FILENAME_FORMAT = './data/yosemite_village/CRNS0101-05-{year}-CA_Yosemite_Village_12_W.txt'

train_temperature_data = pandas.concat([pandas.read_table(FILENAME_FORMAT.format(year=year),
                                                          sep=r'\s+', header=None, names=NAMES)
                                       for year in TRAIN_YEARS])

print(train_temperature_data.shape)
train_temperature_data.head()

(420768, 23)


,WBAN_number,UTC_date,UTC_time,LST_date,LST_time,CRX_version,longitude,latitude,air_temperature,precepitation,...,ST_type,ST_flag,relative_humidity,RH_flag,soil_moisture,soil_temperature,wetness,wet_flag,wind,wind_flag
0,53150,20110101,5,20101231,1605,2,-119.82,37.76,-6.4,0.0,...,U,0,-9999,0,-99.0,-9999.0,1218,0,-99.0,0
1,53150,20110101,10,20101231,1610,2,-119.82,37.76,-6.5,0.0,...,U,0,-9999,0,-99.0,-9999.0,1223,0,-99.0,0
2,53150,20110101,15,20101231,1615,2,-119.82,37.76,-6.5,0.0,...,U,0,-9999,0,-99.0,-9999.0,1223,0,-99.0,0
3,53150,20110101,20,20101231,1620,2,-119.82,37.76,-6.5,0.0,...,U,0,-9999,0,-99.0,-9999.0,1224,0,-99.0,0
4,53150,20110101,25,20101231,1625,2,-119.82,37.76,-6.7,0.0,...,U,0,-9999,0,-99.0,-9999.0,1210,0,-99.0,0


In [4]:
validation_temperature_data = pandas.read_table(FILENAME_FORMAT.format(year=VALIDATION_YEAR),
                                          sep=r'\s+', header=None, names=NAMES)
print(validation_temperature_data.shape)

test_temperature_data = pandas.read_table(FILENAME_FORMAT.format(year=TEST_YEAR),
                                          sep=r'\s+', header=None, names=NAMES)
print(test_temperature_data.shape)

(105120, 23)
(105408, 23)


## Parse only the fields we care about, and clear out NAs
The documentation says -9999 are their NA representation, so we substitute None and drop NAs

In [5]:
TEMPERATURE_PREDICTION_FIELDS = ('LST_date', 'LST_time', 'air_temperature')

limited_train = train_temperature_data.loc[:,TEMPERATURE_PREDICTION_FIELDS]
print(limited_train.shape)
limited_train = limited_train.replace({-9999.0: None}).dropna()
print(limited_train.shape)

limited_validation = validation_temperature_data.loc[:,TEMPERATURE_PREDICTION_FIELDS]
print(limited_validation.shape)
limited_validation = limited_validation.replace({-9999.0: None}).dropna()
print(limited_validation.shape)

limited_test = test_temperature_data.loc[:,TEMPERATURE_PREDICTION_FIELDS]
print(limited_test.shape)
limited_test = limited_test.replace({-9999.0: None}).dropna()
print(limited_test.shape)

(420768, 3)
(420417, 3)
(105120, 3)
(105063, 3)
(105408, 3)
(105374, 3)


## Examine the field we wish to predict, making sure it is reasonably well-behaved

In [6]:
limited_train.hist('air_temperature')
limited_validation.hist('air_temperature')
limited_test.hist('air_temperature')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x11cbc1438>]], dtype=object)

## Parse date to day of year, time to minute of day

In [7]:
DATE_FORMAT = '%Y%m%d'

for dataset in (limited_train, limited_validation, limited_test):
    dataset['LST_day_of_year'] = [datetime.strptime(str(lst_date), DATE_FORMAT).timetuple().tm_yday
                                  for lst_date in dataset.LST_date]

    dataset['LST_time_of_day'] = [lst_time if lst_time < 100 
                                  else int((lst_time % 100) + np.floor(lst_time / 100) * 60)
                                  for lst_time in dataset.LST_time]

## Defining a transformed to apply a pre-provided RBF

In [8]:
from sklearn.base import TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel

class RBFEncoder(TransformerMixin):
    def __init__(self, means, alpha):
        self.means = means if type(means) in (np.array, np.ndarray) else np.array(means).reshape((-1, 1))
        self.alpha = alpha
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # np.exp(- 1 / (2 * self.alpha ** 2) * (X - self.m) ** 2)
#         X = X.reshape(-1, 1)
        return rbf_kernel(X, self.means, gamma=1 / (2 * self.alpha ** 2))

## Seasonal variation (1)
From https://www.nps.gov/yose/planyourvisit/weather.htm

The temperature peak appears to be in July/August, and the trough in December/January, so we can start by computing features as distance within the year from each (day 0, day 212, day 365). We can do something similar with the time of day later.

The hottest time of the day appears to be around 1500 or so.

In [9]:
DAY_MEANS = (0, 212, 365)
DAY_ALPHA = 70
TIME_MEANS = (0, 900, 1440)
TIME_ALPHA = 400


data_mapper = DataFrameMapper([
    (['LST_day_of_year'], RBFEncoder(DAY_MEANS, DAY_ALPHA)),
    (['LST_time_of_day'], RBFEncoder(TIME_MEANS, TIME_ALPHA)),
])

transformed_train = data_mapper.fit_transform(limited_train.copy())
print(transformed_train.shape)
transformed_train[:5]

(420417, 6)


array([[  1.24747661e-06,   9.17513141e-02,   1.00000000e+00,
          5.44714741e-02,   9.86883654e-01,   4.94069974e-01],
       [  1.24747661e-06,   9.17513141e-02,   1.00000000e+00,
          5.28492108e-02,   9.84804140e-01,   5.01419351e-01],
       [  1.24747661e-06,   9.17513141e-02,   1.00000000e+00,
          5.12672504e-02,   9.82575469e-01,   5.08798546e-01],
       [  1.24747661e-06,   9.17513141e-02,   1.00000000e+00,
          4.97248734e-02,   9.80198673e-01,   5.16205674e-01],
       [  1.24747661e-06,   9.17513141e-02,   1.00000000e+00,
          4.82213637e-02,   9.77674853e-01,   5.23638811e-01]])

## Use the transformation defined above to model temperature (2)

In [42]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import Ridge, RidgeCV

np.random.seed(33)


pipe = sklearn.pipeline.Pipeline([
    ('featurize', data_mapper),
    ('ridge', RidgeCV()),
])

def cross_validate():
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        shuffled = limited_train.sample(frac=1)
        cv_scores = np.round(cross_val_score(pipe, X=shuffled, y=shuffled.air_temperature, 
                                             scoring='mean_squared_error', cv=5), 3)
        print(np.average(cv_scores), cv_scores)

cv_time, _ = timeit.timeit(cross_validate, number=3)
        
#     for k in range(3, 16):
#         pipe.set_params(select__k=k)
#         shuffled = data_train.sample(frac=1)
#         cv_scores = np.round(cross_val_score(pipe, X=shuffled, y=shuffled.like, scoring='r2', cv=7), 3)
#         print(k, np.average(cv_scores), cv_scores)

-24.375 [-24.449 -24.353 -24.407 -24.298 -24.368]
-24.3754 [-24.513 -24.292 -24.277 -24.395 -24.4  ]
-24.3748 [-24.322 -24.321 -24.407 -24.333 -24.491]


In [43]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics

def fit():
    pipe.fit(shuffled, shuffled.air_temperature)

shuffled = limited_train.sample(frac=1)
train_time, _ = timeit.timeit(fit, number=3)

def predict():
    return pipe.predict(limited_validation)

predict_time, validation_predictions = timeit.timeit(predict, number=3)

for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    print(metric.__name__, metric(validation_predictions, limited_validation.air_temperature))

mean_squared_error 25.5736733344
mean_absolute_error 4.20662533563
median_absolute_error 3.84212226051
r2_score 0.324454703716


In [46]:
cv_time / 3, train_time / 3, predict_time / 3

(3.1004114066793895, 0.3519575096433982, 0.025259659974835813)

## Plotting different contributions (3)
Under this model, the first three components are the day of year contributions, and the latter three are the time of day, so we can separate them out

Plotting the validation set since it's less massive than the training set

In [12]:
ridge = pipe.steps[1][1]
day_of_year_coef = ridge.coef_[:3]
time_of_day_coef = ridge.coef_[3:]

transformed_validation = data_mapper.transform(limited_validation)

temperature_day_contrib = np.matmul(transformed_validation[:,:3], day_of_year_coef)
temperature_time_contrib = np.matmul(transformed_validation[:,3:], time_of_day_coef)

In [13]:
def plot_contributions(x, y, label):
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, s=5, label=label)
    plt.legend(loc=1, fontsize=16)
    plt.xlabel(label, size=16)
    plt.ylabel(r'Predicted temperatire', size=16)
    plt.show()

In [14]:
plot_contributions(limited_validation.LST_day_of_year, temperature_day_contrib, 'Day of year contribution')
plot_contributions(limited_validation.LST_time_of_day, temperature_time_contrib, 'Time of day contribution')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3-d plot of both time of day and day of year contributions (4)

The color mapping is designed to (maybe?) help understand where the model is over-predicting and where it is under-predicting. I color by the difference between the actual observed temperature and the prediction, coloring red when the actual observed temperature was hotter, white when they're about the same, and blue when the observed temperature was colder.

In [15]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(limited_validation.LST_day_of_year, limited_validation.LST_time_of_day,
          validation_predictions, c=limited_validation.air_temperature - validation_predictions,
          cmap='bwr')

ax.set_xlabel('Day of year', size=12)
ax.set_ylabel('Time of day', size=12)
ax.set_zlabel('Predicted temperature', size=12)

<IPython.core.display.Javascript object>

## Let's comapre time-only and day-only models to above performance (5.1, 5.2)

In [16]:
DAY_MEANS = (0, 212, 365)
DAY_ALPHA = 70
TIME_MEANS = (0, 900, 1440)
TIME_ALPHA = 400


day_only_data_mapper = DataFrameMapper([
    (['LST_day_of_year'], RBFEncoder(DAY_MEANS, DAY_ALPHA)),
])

time_only_data_mapper = DataFrameMapper([
    (['LST_time_of_day'], RBFEncoder(TIME_MEANS, TIME_ALPHA)),
])

day_only_data_mapper.fit_transform(limited_train.copy())
time_only_data_mapper.fit_transform(limited_train.copy())


array([[ 0.05447147,  0.98688365,  0.49406997],
       [ 0.05284921,  0.98480414,  0.50141935],
       [ 0.05126725,  0.98257547,  0.50879855],
       ..., 
       [ 0.05958732,  0.99221794,  0.47221896],
       [ 0.0578398 ,  0.99059142,  0.479468  ],
       [ 0.05613476,  0.98881304,  0.48675226]])

In [17]:
np.random.seed(RANDOM_SEED)

day_only_pipe = sklearn.pipeline.Pipeline([
    ('featurize', day_only_data_mapper),
    ('ridge', RidgeCV()),
])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    shuffled = limited_train.sample(frac=1)
    cv_scores = np.round(cross_val_score(day_only_pipe, X=shuffled, y=shuffled.air_temperature, 
                                         scoring='mean_squared_error', cv=5), 3)
    print(np.average(cv_scores), cv_scores)

    
def fit():
    day_only_pipe.fit(shuffled, shuffled.air_temperature)

shuffled = limited_train.sample(frac=1)
day_only_train_time, _ = timeit.timeit(fit, number=1)

def predict():
    return day_only_pipe.predict(limited_validation)

day_only_predict_time, day_only_validation_predictions = timeit.timeit(predict, number=1)


for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    print(metric.__name__, metric(day_only_validation_predictions, limited_validation.air_temperature))

-26.579 [-26.613 -26.546 -26.687 -26.481 -26.568]
mean_squared_error 28.0590404039
mean_absolute_error 4.3980149437
median_absolute_error 4.03802355552
r2_score 0.21269101949


In [18]:
np.random.seed(RANDOM_SEED)

time_only_pipe = sklearn.pipeline.Pipeline([
    ('featurize', time_only_data_mapper),
    ('ridge', RidgeCV()),
])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    shuffled = limited_train.sample(frac=1)
    cv_scores = np.round(cross_val_score(time_only_pipe, X=shuffled, y=shuffled.air_temperature, 
                                         scoring='mean_squared_error', cv=5), 3)
    print(np.average(cv_scores), cv_scores)

    
def fit():
    time_only_pipe.fit(shuffled, shuffled.air_temperature)

shuffled = limited_train.sample(frac=1)
time_only_train_time, _ = timeit.timeit(fit, number=1)

def predict():
    return time_only_pipe.predict(limited_validation)

time_only_predict_time, time_only_validation_predictions = timeit.timeit(predict, number=1)


for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    print(metric.__name__, metric(time_only_validation_predictions, limited_validation.air_temperature))

-60.0238 [-60.028 -60.442 -59.98  -59.861 -59.808]
mean_squared_error 55.0397262137
mean_absolute_error 6.05221746777
median_absolute_error 5.34623313466
r2_score -24.1436650411


## (5) Conclusion 

The day-only model captures most of the variation, ending with a fairly close MSE to the full model (validation MSE of 28.06, compared to 25.57 for the fully-blown model). On the other hand, the time-of-day model fares miserably, showing an MSE of almost twice as much as the other two, at 55.04 on the validation set.

This makes sense with our first-order understanding of the world; temperature varies by season much more strongly than it varies by the time of day.

## Let's try with a higher number of less-well-engineered basis functions

In [19]:
DAY_MEANS = (0, 61, 122, 183, 244, 365)
DAY_ALPHA = 40
TIME_MEANS = (0, 240, 480, 720, 960, 1200, 1440)
TIME_ALPHA = 100

more_bases_data_mapper = DataFrameMapper([
    (['LST_day_of_year'], RBFEncoder(DAY_MEANS, DAY_ALPHA)),
    (['LST_time_of_day'], RBFEncoder(TIME_MEANS, TIME_ALPHA)),
])

transformed_train = more_bases_data_mapper.fit_transform(limited_train.copy())
print(transformed_train.shape)
transformed_train[:5]

(420417, 13)


array([[  8.30040465e-19,   2.86797501e-13,   9.68378984e-09,
          3.19528238e-05,   1.03030790e-02,   1.00000000e+00,
          6.00766980e-21,   3.85654273e-12,   7.80106730e-06,
          4.97248734e-02,   9.98750781e-01,   6.32127031e-02,
          1.26071052e-05],
       [  8.30040465e-19,   2.86797501e-13,   9.68378984e-09,
          3.19528238e-05,   1.03030790e-02,   1.00000000e+00,
          3.70353198e-21,   2.68054764e-12,   6.11356797e-06,
          4.39369336e-02,   9.95012479e-01,   7.10053537e-02,
          1.59667839e-05],
       [  8.30040465e-19,   2.86797501e-13,   9.68378984e-09,
          3.19528238e-05,   1.03030790e-02,   1.00000000e+00,
          2.27740572e-21,   1.85850261e-12,   4.77913973e-06,
          3.87257704e-02,   9.88813045e-01,   7.95595087e-02,
          2.01712951e-05],
       [  8.30040465e-19,   2.86797501e-13,   9.68378984e-09,
          3.19528238e-05,   1.03030790e-02,   1.00000000e+00,
          1.39694394e-21,   1.28533723e-12,   3.726

In [20]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import Ridge, RidgeCV

np.random.seed(33)

more_bases_pipe = sklearn.pipeline.Pipeline([
    ('featurize', more_bases_data_mapper),
    ('ridge', RidgeCV()),
])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    shuffled = limited_train.sample(frac=1)
    cv_scores = np.round(cross_val_score(more_bases_pipe, X=shuffled, y=shuffled.air_temperature, 
                                         scoring='mean_squared_error', cv=5), 3)
    print(np.average(cv_scores), cv_scores)
    
#     for k in range(3, 16):
#         pipe.set_params(select__k=k)
#         shuffled = data_train.sample(frac=1)
#         cv_scores = np.round(cross_val_score(pipe, X=shuffled, y=shuffled.like, scoring='r2', cv=7), 3)
#         print(k, np.average(cv_scores), cv_scores)

-23.6812 [-23.759 -23.652 -23.684 -23.615 -23.696]


In [21]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics

def fit():
    more_bases_pipe.fit(shuffled, shuffled.air_temperature)

shuffled = limited_train.sample(frac=1)
more_bases_train_time, _ = timeit.timeit(fit, number=1)

def predict():
    return more_bases_pipe.predict(limited_validation)

time_only_predict_time, more_bases_validation_predictions = timeit.timeit(predict, number=1)

for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    print(metric.__name__, metric(more_bases_validation_predictions, 
                                  limited_validation.air_temperature))

mean_squared_error 25.5417811586
mean_absolute_error 4.18470465283
median_absolute_error 3.78349262737
r2_score 0.337450520671


## Adding more basis functions helps

Shcoking conclusion, truly. When you give a linear model more data, it does better... 

Let's do two things (one above, one below). 

Above, I'll add a validation set, using the 2015 data, to make sure the improvements are not training-only.

### Note: validation set performance looks identical (scarily so) between the two. Very weird.

Had I had more time, I would have liked to be more Bayesian about it. Perhaps use model evidence or something to make sure that I'm actually gaining performance from the additional bases. I'll have to see if I can get ridge models to give me evidence or BIC, or if not, try something else. The quick answer is that I cannot get the model evidence or BIC directly.

I'd also like to be searching over the different basis means, rather than hard-coding them, as if I can learn them from the data I should arrive at better results.

I'll try to fit a Gaussian mixture model using the dirichlet process below, and see if I can get some better idea for what sort of means I'd like to use. 

In [22]:
from sklearn.mixture import BayesianGaussianMixture

FIELDS = ('LST_day_of_year', 'LST_time_of_day', 'air_temperature')
mixture_train = limited_train.loc[:,FIELDS]

mixture = BayesianGaussianMixture(n_components=20, max_iter=1000, verbose=1)
mixture.fit(mixture_train)

Initialization 0
  Iteration 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
  Iteration 60
  Iteration 70
  Iteration 80
  Iteration 90
  Iteration 100
  Iteration 110
  Iteration 120
  Iteration 130
  Iteration 140
  Iteration 150
  Iteration 160
  Iteration 170
  Iteration 180
  Iteration 190
  Iteration 200
  Iteration 210
  Iteration 220
  Iteration 230
  Iteration 240
  Iteration 250
  Iteration 260
  Iteration 270
  Iteration 280
  Iteration 290
  Iteration 300
  Iteration 310
  Iteration 320
  Iteration 330
  Iteration 340
  Iteration 350
  Iteration 360
  Iteration 370
  Iteration 380
  Iteration 390
  Iteration 400
  Iteration 410
  Iteration 420
  Iteration 430
  Iteration 440
  Iteration 450
  Iteration 460
  Iteration 470
  Iteration 480
  Iteration 490
  Iteration 500
  Iteration 510
  Iteration 520
  Iteration 530
  Iteration 540
  Iteration 550
  Iteration 560
  Iteration 570
  Iteration 580
  Iteration 590
  Iteration 600
  Iteration 610
  

 /Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning:Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.


BayesianGaussianMixture(covariance_prior=None, covariance_type='full',
            degrees_of_freedom_prior=None, init_params='kmeans',
            max_iter=1000, mean_precision_prior=None, mean_prior=None,
            n_components=20, n_init=1, random_state=None, reg_covar=1e-06,
            tol=0.001, verbose=1, verbose_interval=10, warm_start=False,
            weight_concentration_prior=None,
            weight_concentration_prior_type='dirichlet_process')

In [23]:
print(mixture.weights_)
day_and_time_means = mixture.means_[:,:2]
day_and_time_gamma_estimate = np.sqrt(np.mean(mixture.covariances_[:,:2,:2]))

[ 0.08845834  0.0311718   0.01917936  0.06514806  0.04609655  0.08173242
  0.07504534  0.02877392  0.04402773  0.03930735  0.09267037  0.05854162
  0.058619    0.06068031  0.02426741  0.04044645  0.04028055  0.04183969
  0.01999399  0.04371972]


In [24]:
combined_data_mapper = DataFrameMapper([
    (['LST_day_of_year', 'LST_time_of_day'], RBFEncoder(day_and_time_means, day_and_time_gamma_estimate)),
])

combined_transformed_train = combined_data_mapper.fit_transform(limited_train.copy())
print(combined_transformed_train.shape)
combined_transformed_train[:5]

(420417, 20)


array([[  1.85727959e-09,   2.12801748e-04,   7.75069717e-21,
          2.23474347e-01,   6.28505996e-02,   3.26145216e-05,
          5.02443983e-05,   2.58176419e-06,   2.87323961e-14,
          6.74540517e-02,   2.21330862e-02,   3.89824136e-06,
          1.89026590e-03,   6.40442833e-02,   4.18032112e-22,
          3.38197387e-04,   3.76413204e-02,   2.58252732e-16,
          1.88644716e-19,   2.01535893e-09],
       [  1.37061146e-09,   2.60896204e-04,   4.70878591e-21,
          2.33813923e-01,   5.58098741e-02,   2.60140509e-05,
          4.22414158e-05,   3.23439568e-06,   1.90622311e-14,
          5.96517826e-02,   2.07782054e-02,   3.25218014e-06,
          2.15267147e-03,   7.20094431e-02,   2.53943861e-22,
          3.77530113e-04,   3.29047967e-02,   1.68767198e-16,
          1.15898653e-19,   1.44684821e-09],
       [  1.00869225e-09,   3.18983030e-04,   2.85288569e-21,
          2.43960948e-01,   4.94219538e-02,   2.06924640e-05,
          3.54157572e-05,   4.04088948e-06

In [25]:
np.random.seed(RANDOM_SEED)

combined_means_pipe = sklearn.pipeline.Pipeline([
    ('featurize', combined_data_mapper),
    ('ridge', RidgeCV()),
])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    shuffled = limited_train.sample(frac=1)
    cv_scores = np.round(cross_val_score(combined_means_pipe, X=shuffled, y=shuffled.air_temperature, 
                                         scoring='mean_squared_error', cv=5), 3)
    print(np.average(cv_scores), cv_scores)

    
shuffled = limited_train.sample(frac=1)
combined_means_pipe.fit(shuffled, shuffled.air_temperature)

combined_validation_predictions = combined_means_pipe.predict(limited_validation)

for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    print(metric.__name__, metric(combined_validation_predictions, limited_validation.air_temperature))

-30.4432 [-30.413 -30.386 -30.551 -30.453 -30.413]
mean_squared_error 32.6245838862
mean_absolute_error 4.69254419145
median_absolute_error 4.19885920521
r2_score -0.0260554005082


## Running the first (simplest) model on the test set

In [26]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics

shuffled = limited_train.sample(frac=1)
pipe.fit(shuffled, shuffled.air_temperature)

temperature_test_predictions = pipe.predict(limited_test)

for metric in (metrics.mean_squared_error, metrics.mean_absolute_error, 
               metrics.median_absolute_error, metrics.r2_score): 
    print(metric.__name__, metric(temperature_test_predictions, limited_test.air_temperature))

mean_squared_error 22.3595903914
mean_absolute_error 3.91367291816
median_absolute_error 3.5469636434
r2_score 0.410237154425


The test MSE is even lower than the validation one. Cool!

# Classification problem - predict rainfall
Relevant fields:
* date and time, as parsed above, perhaps even with the basis functions
* air temperature (we ignore surface temperature as they're likely similar?)
* precipitation itself, which we need to convert to a binary indicator
* solar radiation
    * removing rows where sr_flag is not 0 (overflow or erroneous data)
* relative humidity, making sure it's not too strong a predictor (since rain might == 100% humidity?)
    * and similarly, removing rows where rh_flag is not 0
* soil_moisture, under the same note for relative humidity (if it's raining soil is bound to be moist?)
* by their definition, wetness is a very strong predictor for precipitation, which means that for the model to be useful we should probably omit it
* wind_1_5, the wind speed at 1.5
    * again, remove rows where wind_flag is not 0

In [27]:
RAINFALL_PREDICTION_FIELDS = ('LST_date', 'LST_time', 'air_temperature', 'precepitation',
                             'solar_radiation', 'SR_flag', 'relative_humidity', 'RH_flag',
                             'soil_moisture', 'wind', 'wind_flag')

rain_train = train_temperature_data.loc[:,RAINFALL_PREDICTION_FIELDS]
print(rain_train.shape)
rain_train = rain_train.replace({-9999.0: None}).dropna()
print(rain_train.shape)
for flag_field in ('SR_flag', 'RH_flag', 'wind_flag'):
    rain_train[flag_field] = rain_train[flag_field].replace({1: None, 3: None})
rain_train = rain_train.dropna()
print(rain_train.shape)
print()

rain_validation = validation_temperature_data.loc[:,RAINFALL_PREDICTION_FIELDS]
print(rain_validation.shape)
rain_validation = rain_validation.replace({-9999.0: None}).dropna()
print(rain_validation.shape)
for flag_field in ('SR_flag', 'RH_flag', 'wind_flag'):
    rain_validation[flag_field] = rain_validation[flag_field].replace({1: None, 3: None})
rain_validation = rain_validation.dropna()
print(rain_validation.shape)
print()

rain_test = test_temperature_data.loc[:,RAINFALL_PREDICTION_FIELDS]
print(rain_test.shape)
rain_test = rain_test.replace({-9999.0: None}).dropna()
print(rain_test.shape)
for flag_field in ('SR_flag', 'RH_flag', 'wind_flag'):
    rain_test[flag_field] = rain_test[flag_field].replace({1: None, 3: None})
rain_test = rain_test.dropna()
print(rain_test.shape)
print()


(420768, 11)
(364072, 11)
(364072, 11)

(105120, 11)
(105063, 11)
(105063, 11)

(105408, 11)
(105374, 11)
(105374, 11)



## Let's examine the covariance matrix quickly

Looking at the validation set, since it's smaller.

Plotting code inspired by https://stackoverflow.com/questions/40887753/display-matrix-values-and-colormap

Plotting the log-covariance with the original sign, to shrink the values a little. Not super useful, but still neat.

In [28]:
from sklearn.covariance import empirical_covariance

cov = empirical_covariance(rain_validation)
log_cov = np.log(np.abs(cov) + 1) * np.sign(cov)

import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

min_val, max_val = np.min(log_cov), np.max(log_cov)

ax.matshow(log_cov, cmap=plt.cm.Blues)

# for i in range(log_cov.shape[1]):
#     for j in range(log_cov.shape[0]):
#         c = log_cov[j, i]
#         ax.text(i, j, str(c), va='center', ha='center')

<IPython.core.display.Javascript object>

## Let's plot precipitation to understand how I should binarize it

Do I even want to binarize it, though? That loses information, which is unfortunate (i.e. on a timestamp with more rain I'd want to predict rain with higher certainty?). Perhaps I'd rather regress linearly and then decide how to apply a non-linearity to transform to the range 0, 1?

In [29]:
rain_train.hist('precepitation')

<IPython.core.display.Javascript object>

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x11d73e748>]], dtype=object)

The above does not look too promising. Let's investigate the data further:

In [30]:
values = set(rain_train.precepitation)
for rain in sorted(values):
    print(rain, np.sum(rain_train.precepitation == rain))
    
print(np.sum(rain_train.precepitation == 0) / rain_train.shape[0])

0.0 356685
0.2 3240
0.3 1992
0.4 908
0.5 495
0.6 263
0.7 185
0.8 107
0.9 58
1.0 46
1.1 27
1.2 24
1.3 8
1.4 5
1.5 7
1.6 3
1.7 2
1.8 2
1.9 2
2.0 2
2.5 2
2.6 2
2.7 1
2.8 1
3.1 1
3.6 1
3.8 1
4.5 1
5.3 1
0.979710057351


## My fears about binarization were premature (2)
Since the vast majority of timestamps have a rain value of 0, any model we train will be biased towards predicting 0, since it can receive an accuracy of almost 98% by just predicting false all the time.

This means I'll have to reach into the class imbalance arsenal - providing a substantial class weight for positives, oversampling them, or both. 

In [31]:
rain_train.precepitation = rain_train.precepitation > 0
rain_validation.precepitation = rain_validation.precepitation > 0
rain_test.precepitation = rain_test.precepitation > 0

In [32]:
DATE_FORMAT = '%Y%m%d'

for dataset in (rain_train, rain_validation, rain_test):
    dataset['LST_day_of_year'] = [datetime.strptime(str(lst_date), DATE_FORMAT).timetuple().tm_yday
                                  for lst_date in dataset.LST_date]

    dataset['LST_time_of_day'] = [lst_time if lst_time < 100 
                                  else int((lst_time % 100) + np.floor(lst_time / 100) * 60)
                                  for lst_time in dataset.LST_time]

## To normalize the non-basis functions, I use the standard scaler

In [33]:
DAY_MEANS = (0, 212, 365)
DAY_ALPHA = 70
TIME_MEANS = (0, 900, 1440)
TIME_ALPHA = 400

rain_data_mapper = DataFrameMapper([
    ('LST_day_of_year', RBFEncoder(DAY_MEANS, DAY_ALPHA)),
    ('LST_time_of_day', RBFEncoder(TIME_MEANS, TIME_ALPHA)),
    (['air_temperature'], sklearn.preprocessing.StandardScaler()),
    (['solar_radiation'], sklearn.preprocessing.StandardScaler()),
    (['relative_humidity'], sklearn.preprocessing.StandardScaler()),
    (['soil_moisture'], sklearn.preprocessing.StandardScaler()),
    (['wind'], sklearn.preprocessing.StandardScaler()),
])

transformed_train = rain_data_mapper.fit_transform(rain_train.copy())
print(transformed_train.shape)
transformed_train[:5]

 /Users/guydavidson/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning:Data with input dtype int64 was converted to float64 by StandardScaler.


ValueError: Expected 2D array, got 1D array instead:
array=[ 196.  196.  196. ...,  365.  365.  365.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Train a logistic regression model, cross-validating to select parameters
We perform a random search CV to pick the number of features to use, as well as the C (penalty) and positive class weight for the logistic regression model. The number of features is searched in a uniform distribution over integers, and C and the class weight are searched in log scale, so sampling uniformly and raising by the exponent.

Much of the code below is copied from my week 2 classification work

In [ ]:
# report function from http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

np.random.seed(RANDOM_SEED)

logit_search_pipe = sklearn.pipeline.Pipeline([
    ('featurize', rain_data_mapper),
    ('select', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression)),
    ('logit', sklearn.linear_model.LogisticRegression(verbose=True))
])

from collections import namedtuple
ClassWeightSampler = namedtuple('ClassWeightSampler', 'rvs')
CSampler = namedtuple('CSampler', 'rvs')

def sample_class_weights(loc=0, scale=1, size=1, random_state=None):
    return {0: 1, 1: np.exp(uniform.rvs(loc=0, scale=4, size=size, random_state=random_state))[0]}


def log_sample_cv(loc=0, scale=1, size=1, random_state=None):
    return np.exp(uniform.rvs(loc=0, scale=4, size=size, random_state=random_state))[0]

class_weight_sampler = ClassWeightSampler(rvs=sample_class_weights)
c_sampler = CSampler(rvs=log_sample_cv)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    search_cv = RandomizedSearchCV(logit_search_pipe, n_iter=20, scoring='f1', n_jobs=4, 
                                   verbose=1, random_state=RANDOM_SEED,
                                   param_distributions={'select__k': randint(3, 10),
                                                        'logit__class_weight': class_weight_sampler,
                                                        'logit__C': c_sampler 
                                                       })
    search_cv.fit(X=rain_train, y=rain_train.precepitation)
    report(search_cv.cv_results_)



In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, mutual_info_score

validation_predictions = search_cv.best_estimator_.predict(rain_validation)

for metric in (classification_report, accuracy_score, roc_auc_score, mutual_info_score):
    print(metric.__name__)
    print(metric(validation_predictions, rain_validation.precepitation))


The results look reasonable - the recall for true examples is still terrible, but on the balance it appears reasonable. The validation f1 score is better than on the training set, which is solid.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, mutual_info_score

test_predictions = search_cv.best_estimator_.predict(rain_test)

for metric in (classification_report, accuracy_score, roc_auc_score, mutual_info_score):
    print(metric.__name__)
    print(metric(test_predictions, rain_test.precepitation))
